In [ ]:
import os
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

def load_images_from_path(path, label):
    images = []
    column_labels = []

    for file in os.listdir(path):
        img = image.load_img(os.path.join(path, file), target_size=(224, 224, 3))
        images.append(image.img_to_array(img))
        column_labels.append((label))
        
    return images, column_labels
def show_images(images):
    fig, axes = plt.subplots(1, 8, figsize=(20, 20), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i] / 255)

x_train = []
y_train = []
x_test = []
y_test = []

Arctic-fox training images - Initial Code to show few artic fox images

In [ ]:
images, column_labels = load_images_from_path('Data/Wildlife/train/arctic_fox', 0)
show_images(images)
    
x_train += images
y_train += column_labels

Polar Bear training images - Initial Code to show few polar bear images

In [ ]:
images, column_labels = load_images_from_path('Data/Wildlife/train/polar_bear', 1)
show_images(images)
    
x_train += images
y_train += column_labels

Walrus training images - Initial Code to show few walrus images

In [ ]:
images, column_labels = load_images_from_path('Data/Wildlife/train/walrus', 2)
show_images(images)

x_train += images
y_train += column_labels

Arctic-fox test images - Initial Code to show few artic fox images

In [ ]:
images, column_labels = load_images_from_path('Data/Wildlife/test/arctic_fox', 0)
show_images(images)
    
x_test += images
y_test += column_labels

Polar Bear test images - Initial Code to show few polar bear images

In [ ]:
images, column_labels = load_images_from_path('Data/Wildlife/test/polar_bear', 1)
show_images(images)
    
x_test += images
y_test += column_labels

Walrus test images - Initial Code to show few walrus images

In [ ]:
images, column_labels = load_images_from_path('Data/Wildlife/test/walrus', 2)
show_images(images)
    
x_test += images
y_test += column_labels

Preprocess the training and testing images, and use Keras's [`to_categorical`](https://keras.io/api/utils/python_utils/#to_categorical-function) function to one-hot-encode the training and testing labels.

In [ ]:
from tensorflow.keras.utils import to_categorical

x_train = np.array(x_train) / 255
x_test = np.array(x_test) / 255

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

## Build and train a CNN

Keras makes it easy to build and train CNNs using classes such as [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/#conv2d-class) and [MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/#maxpooling2d-class). Let's build a CNN with four convolutional layers, followed by a pair of fully connected layers for classification.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Train the model by calling [`fit`](https://keras.io/api/models/model_training_apis/#fit-method).

In [ ]:
hist = model.fit(x_train, y_train_encoded, validation_data=(x_test, y_test_encoded), batch_size=10, epochs=10)

How well did the CNN train? Let's plot the training accuracy and validation accuracy for each epoch.

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, '-', label='Training Accuracy')
plt.plot(epochs, val_acc, ':', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()

The validation accuracy is decent, but it's not state-of-the art. Modern CNNs often achieve an accuracy of 95% or more doing image classification. You could increase the accuracy by increasing the number of training images by an order of magnitude or more, but training time would increase commensurately. One solution for that is to do the training on an HPC cluster outfitted with GPUs. A more practical solution is [transfer learning](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a), which enables sophisticated image-classification networks to be trained on an ordinary CPU.